#1.Import Librarys


**1.1 Importing necassary librarires and packages**

In [ ]:
import pandas as pd #linear algebra
import numpy as np #data processing 
import matplotlib.pyplot as plt #plot graphs

**1.2 Importing various sklearn packages**


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,auc,roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize,LabelEncoder
from sklearn.metrics import precision_recall_curve,average_precision_score



**1.3 Importing various plotly packages**

In [ ]:
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

**1.4 Importing various Tenserflow Packages**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras import backend as K


import tensorflow as tf

#2.Prepare Data



**2.1 Load Data**

using pandas to load our Sample Data  

In [ ]:
df = pd.read_csv('SampleNormalized&Encoded.csv')#load Data 
features = df.drop("Unnamed: 0",axis=1)#drop the indicies Column 
features.head()

,Severity,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,...,Zipcode_0,Zipcode_1,Zipcode_2,Zipcode_3,Zipcode_4,Zipcode_5,Zipcode_6,Zipcode_7,Zipcode_8,Zipcode_9
0,1,0.0,0.927273,0.948187,0.212121,0.875163,0.1250,0.043103,0.000,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0.0,0.878788,0.906736,0.181818,0.875489,0.1250,0.043103,0.000,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0.0,0.715152,0.766839,0.929293,0.940026,0.1125,0.021552,0.002,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0.0,0.896970,0.922280,0.171717,0.911995,0.1250,0.021552,0.000,0,...,0,0,0,0,0,0,0,0,1,1
4,1,0.0,0.878788,0.906736,0.212121,0.964798,0.1250,0.038793,0.000,0,...,0,0,0,0,0,0,0,1,0,0


**2.2 Split data Train/Valid/Test**

In [ ]:
X = features.drop("Severity", axis=1)
Labels = features["Severity"]

train_x,test_x,train_y,test_y = train_test_split(X, Labels,shuffle=True, test_size = 0.2, random_state=2)
test_x,valid_x,test_y,valid_y = train_test_split(test_x,test_y, test_size = 0.5,random_state=2)

print(train_x.shape , test_x.shape , valid_x.shape)

(72092, 54) (9012, 54) (9012, 54)


**2.3 Declare Dictionaries to save results**

  we will create 4 dict to save the accuracy , f1_score , recall and precision of each model.

In [ ]:
accuracy_d = dict()
precision_d = dict()
recall_d = dict()
f1_d = dict() 

#3. Neural Network


  Is defined as a black box that takes one or multiple inputs and processes them into one or multiple outputs.

   it consists of many small units called “neurons” grouped into several layers, units of one layer interact with the Neurons of the next layer through "weighted connections" which are connections with a real-valued number attached to them.

  A neuron takes the value of a connected neuron and multiplies it with its connections weight. The sum of all connected neurons and the neuron's bias value is the “activation function", which transforms the value before can be passed on to the next neuron. The idea is to find the right weights to get the right results.

   *In this section* we are going to train a neural network to be able predict the severity of an accident   



  ***3.1 Prepare Data to fit into a Neural Network***

In [ ]:
y = Labels

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
y = to_categorical(y)
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X,y, test_size = 0.3,shuffle=True, random_state = 101)
print(X_train_nn.shape , X_test_nn.shape)


(63081, 54) (27035, 54)


  ***3.2 various Metrics***

here we will manual calculate recall,precision and f1_score because they are not available in keras anymore we will need them later to evaluate 

In [ ]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

***3.3 Create model And fit data**

In [ ]:
model = Sequential()
model.add(Dense(50, activation = 'relu', input_dim = len(features.columns) - 1))
model.add(Dense(15 , activation = 'relu'))
model.add(Dense(15 , activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',optimizer='nadam',
              metrics=['accuracy',f1,precision, recall])

In [ ]:
model.fit(X_train_nn, y_train_nn, epochs = 30,batch_size = 64, 
          validation_data = (X_test_nn, y_test_nn))

Epoch 1/30
986/986 [==============================] - 9s 6ms/step - loss: 0.9828 - accuracy: 0.5776 - f1: 0.4673 - precision: 0.6817 - recall: 0.3762 - val_loss: 0.8859 - val_accuracy: 0.6322 - val_f1: 0.5863 - val_precision: 0.6905 - val_recall: 0.5106
Epoch 2/30
986/986 [==============================] - 9s 9ms/step - loss: 0.8281 - accuracy: 0.6571 - f1: 0.6208 - precision: 0.7220 - recall: 0.5461 - val_loss: 0.7982 - val_accuracy: 0.6705 - val_f1: 0.6403 - val_precision: 0.7424 - val_recall: 0.5641
Epoch 3/30
986/986 [==============================] - 10s 10ms/step - loss: 0.7581 - accuracy: 0.6820 - f1: 0.6595 - precision: 0.7357 - recall: 0.5990 - val_loss: 0.7414 - val_accuracy: 0.6858 - val_f1: 0.6647 - val_precision: 0.7345 - val_recall: 0.6078
Epoch 4/30
986/986 [==============================] - 6s 6ms/step - loss: 0.7340 - accuracy: 0.6904 - f1: 0.6703 - precision: 0.7384 - recall: 0.6147 - val_loss: 0.7327 - val_accuracy: 0.6878 - val_f1: 0.6749 - val_precision: 0.7188 - v

***3.4 Evaluate Model***

In [ ]:
loss, accuracy, f1, precision, recall = model.evaluate(X_test_nn, y_test_nn)
print("Current accuracy is {:.2f}%".format(accuracy*100,".2f"))

845/845 [==============================] - 2s 3ms/step - loss: 0.6782 - accuracy: 0.7111 - f1: 0.6943 - precision: 0.7553 - recall: 0.6441
Current accuracy is 71.11%


In [ ]:
accuracy_d["neural Networks"] = accuracy
precision_d["neural Networks"] = precision 
recall_d ["neural Networks"] = recall
f1_d ["neural Networks"] = f1


#4 Random Forest

 

  Supervised Machine Learning Algorithm, It builds decision trees at training time on different samples and takes their majority vote for classification and average in case of regression. It can handle the data set containing continuous variables as in the case of regression and categorical variables as in the case of classification. It performs better results for classification problems.
  
  after training a neural network in the previous section now it's time to go to Machine learning and since Tensorflow is oriented more to Deep learning we will use Sci-kit learn from now on

  ***4.1 Select the best HyperParamater***

In [ ]:
ranForest = RandomForestClassifier(random_state=2)
hyperPar = [{"criterion": ["gini", "entropy"],
             "n_estimators": [10,50,100 ],"max_depth": [2,5,10],
             }]
grid = GridSearchCV(ranForest, hyperPar, n_jobs=-1, verbose=1)
result = grid.fit(train_x, train_y)
print('Best Hyperparameters: %s' % result.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 100}


***4.2 Fit Data in the model***
  
  we will use the best HyperParamater get from above 

In [ ]:
ranForest =  RandomForestClassifier(random_state=2,criterion="gini",max_depth=10,n_estimators=100)
ranForest.fit(train_x,train_y)
print("accuracy with best HyperParameters")
score = ranForest.score(train_x, train_y)
print("accuracy = {:.2f}%".format(score*100,".2f"))
val_score = ranForest.score(valid_x, valid_y)
print("val_accuracy = {:.2f}%".format(val_score*100,".2f"))

accuracy with best HyperParamaters
accuracy = 74.37%
val_accuracy = 72.38%


***4.3 Classification Report***

In [ ]:
pred_y = ranForest.predict(test_x)
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           1       0.81      0.98      0.88      2256
           2       0.78      0.75      0.76      2315
           3       0.61      0.69      0.65      2235
           4       0.67      0.46      0.55      2206

    accuracy                           0.72      9012
   macro avg       0.72      0.72      0.71      9012
weighted avg       0.72      0.72      0.71      9012



In [ ]:
accuracy_d["Random Forest"] = score
f1_d["Random Forest"] = f1_score(test_y, pred_y,average='micro')


***4.4 Confusion Matrix***

In [ ]:
cf_matrix = confusion_matrix(y_true=test_y, y_pred=pred_y)
cf_matrix 

array([[2201,    1,   21,   33],
       [ 101, 1727,  361,  126],
       [ 319,   30, 1544,  342],
       [ 102,  463,  626, 1015]])

In [ ]:
y = ["Actual Severity 1", "Actual Severity 2",
     "Actual Severity 3", "Actual Severity 4"]
x =  ["Predicted Severity 1", "Predicted Severity 2",
      "Predicted Severity 3", "Predicted Severity 4"]

# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in cf_matrix]

# set up figure 
fig = ff.create_annotated_heatmap(cf_matrix, x=x, y=y, annotation_text=z_text,
                                  colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.5,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.5,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        #textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(yaxis = dict(categoryorder = 'category descending'))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()
fig.write_html("Confusion_Matrix_Random_Forest.html")

***4.5 PR Curve***

In [ ]:
True_pos_Ratio = dict()
False_pos_Ratio = dict()

In [ ]:
Y = label_binarize(valid_y, classes=[1, 2, 3, 4])

y_score = ranForest.predict_proba(valid_x)

precision_d["Random Forest"], recall_d["Random Forest"], _ = precision_recall_curve(Y.ravel(), y_score.ravel())
False_pos_Ratio["Random Forest"], True_pos_Ratio["Random Forest"], _ = roc_curve(Y.ravel(), y_score.ravel())

#5 Logistic Regression 

Classic statistical model for classification to predict a binary output based on prior observations of a data set, it suppresses the output of the linear model between 0 and 1 so the output is a probability value of whether the inputs belong to class 0 or class 1


***5.1 Select the best HyperParamater***

In [ ]:
model = LogisticRegression()
# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = [ 'l2']

...
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
randm_src = RandomizedSearchCV(model, space, n_iter=3, scoring='accuracy'
, n_jobs=-1, cv=cv, random_state=1)
# execute search
result = randm_src.fit(train_x, train_y)
# summarize result

print('Best Hyperparameters: %s' % result.best_params_)


Best Hyperparameters: {'solver': 'newton-cg', 'penalty': 'l2'}


***5.2 Fit Data in the model***
  
  we will use the best HyperParamater get from above 

In [ ]:
logReg = LogisticRegression(solver = "sag",penalty="l2", random_state=2)
logReg.fit(train_x,train_y)
print("accuracy with best HyperParameters")
score = logReg.score(train_x, train_y)
print("accuracy = {:.2f}%".format(score*100,".2f"))
val_score = logReg.score(valid_x, valid_y)
print("val_accuracy = {:.2f}%".format(val_score*100,".2f"))

accuracy with best HyperParamaters
accuracy = 62.90%
val_accuracy = 63.24%


***5.3 Classification Report***

In [ ]:
pred_y = logReg.predict(test_x)
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           1       0.69      0.87      0.77      2256
           2       0.65      0.62      0.63      2315
           3       0.56      0.61      0.58      2235
           4       0.58      0.41      0.48      2206

    accuracy                           0.63      9012
   macro avg       0.62      0.63      0.62      9012
weighted avg       0.62      0.63      0.62      9012



In [ ]:
accuracy_d["Logistic Regression"] = score
f1_d["Logistic Regression"] = f1_score(test_y, pred_y,average='micro')

***5.4 Confusion Matrix***

In [ ]:
cf_matrix = confusion_matrix(y_true=test_y, y_pred=pred_y)
cf_matrix 

array([[1969,  128,   98,   61],
       [ 364, 1434,  324,  193],
       [ 281,  198, 1354,  402],
       [ 233,  451,  627,  895]])

In [ ]:
y = ["Actual Severity 1", "Actual Severity 2", "Actual Severity 3", "Actual Severity 4"]
x =  ["Predicted Severity 1", "Predicted Severity 2", "Predicted Severity 3", "Predicted Severity 4"]

# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in cf_matrix]

# set up figure 
fig = ff.create_annotated_heatmap(cf_matrix, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.5,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.5,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        #textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(yaxis = dict(categoryorder = 'category descending'))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()




fig.write_html("Confusion_Matrix_Logistic_Regression.html")

***5.5 PR Curve***

In [ ]:
Y = label_binarize(valid_y, classes=[1, 2, 3, 4])

y_score = logReg.predict_proba(valid_x)

precision_d["Logistic Regression"], recall_d["Logistic Regression"], _ = precision_recall_curve(Y.ravel(), y_score.ravel())
False_pos_Ratio["Logistic Regression"], True_pos_Ratio["Logistic Regression"], _ = roc_curve(Y.ravel(), y_score.ravel())

#6 Decision Trees

Supervised statistical, Machine Learning, and Data Mining Algorithm, it builds classification or regression models and it organizes a series of roots in a Tree Structure, where the data is continuously split according to a certain parameter. It splits a dataset into smaller subsets, while at the same time an associated decision tree is incrementally developed. The final outcome is a tree with decision nodes (make any decision and have multiple branches) and leaf nodes(the output of those decisions and do not contain any further branches).


***6.1 Select the best HyperParamater***

In [ ]:
decTree = DecisionTreeClassifier(random_state=2)
hyperPar = [{"criterion": ["gini", "entropy"],
             "splitter": ["best", "random"],"max_depth": [10,20,30],
             "max_features" : ["auto", "sqrt", "log2"]}]
grid = GridSearchCV(decTree, hyperPar, n_jobs=-1, verbose=1)
result = grid.fit(train_x, train_y)
print('Best Hyperparameters: %s' % result.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


90 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'


/usr/local/lib/python3.7/dist-pack

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto', 'splitter': 'best'}


***6.2 Fit Data in the model***
  
  we will use the best HyperParamater get from above 

In [ ]:
decTree = DecisionTreeClassifier(random_state=2,criterion="entropy",max_depth=10,max_features="auto",splitter="best")
decTree.fit(train_x,train_y)
print("accuracy with best HyperParamaters")
score = decTree.score(train_x, train_y)
print("accuracy = {:.2f}%".format(score*100,".2f"))
val_score = logReg.score(valid_x, valid_y)
print("val_accuracy = {:.2f}%".format(val_score*100,".2f"))

accuracy with best HyperParamaters
accuracy = 65.14%
val_accuracy = 63.24%


***6.3 Classification Report***

In [ ]:
pred_y = decTree.predict(test_x)
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           1       0.74      0.93      0.82      2256
           2       0.72      0.64      0.68      2315
           3       0.51      0.64      0.57      2235
           4       0.56      0.33      0.42      2206

    accuracy                           0.64      9012
   macro avg       0.63      0.64      0.62      9012
weighted avg       0.63      0.64      0.62      9012



In [ ]:
accuracy_d["Decision Tree"] = score
f1_d["Decision Tree"] = f1_score(test_y, pred_y,average='weighted')

***6.4 Confusion Matrix***

In [ ]:
cf_matrix = confusion_matrix(y_true=test_y, y_pred=pred_y)
cf_matrix 

array([[2100,    6,  126,   24],
       [ 170, 1487,  436,  222],
       [ 360,  114, 1427,  334],
       [ 219,  461,  792,  734]])

In [ ]:
y = ["Actual Severity 1", "Actual Severity 2", "Actual Severity 3", "Actual Severity 4"]
x =  ["Predicted Severity 1", "Predicted Severity 2", "Predicted Severity 3", "Predicted Severity 4"]

# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in cf_matrix]

# set up figure 
fig = ff.create_annotated_heatmap(cf_matrix, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.5,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.5,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        #textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(yaxis = dict(categoryorder = 'category descending'))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()




fig.write_html("Confusion_Matrix_Decision_Tree.html")

***6.5 PR Curve***

In [ ]:
Y = label_binarize(valid_y, classes=[1, 2, 3, 4])

y_score = logReg.predict_proba(valid_x)

precision_d["Decision Tree"], recall_d["Decision Tree"], _ = precision_recall_curve(Y.ravel(), y_score.ravel())
False_pos_Ratio["Decision Tree"], True_pos_Ratio["Decision Tree"], _ = roc_curve(Y.ravel(), y_score.ravel())

***6.6 ROC curve*** 

# 7 naive Bayes

Supervised probabilistic classification. It's based on applying Bayes’ theorem with the “naive” hypothesis of conditional independence between every pair of features given the value of the class variable, for example, finding the probability of A happening, given that B has occurred.

***7.1 Select the best HyperParamater***

In [ ]:
clf = MultinomialNB()
hyperPar = [{"fit_prior": [True,False]}]
grid = GridSearchCV(clf, hyperPar, n_jobs=-1, verbose=1)
result = grid.fit(train_x, train_y)

print('Best Hyperparameters: %s' % result.best_params_)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best Hyperparameters: {'fit_prior': False}


***7.2 Fit Data in the model***
  
  we will use the best HyperParamater get from above 

In [ ]:
clf.fit(train_x,train_y)
print("accuracy with best HyperParamaters")
score = clf.score(train_x, train_y)
print("accuracy = {:.2f}%".format(score*100,".2f"))
val_score = clf.score(valid_x, valid_y)
print("val_accuracy = {:.2f}%".format(val_score*100,".2f"))

accuracy with best HyperParamaters
accuracy = 48.91%
val_accuracy = 48.62%


***7.3 Classification Report***

In [ ]:
pred_y = clf.predict(test_x)

accuracy_d["CategoricalNB"] = score
f1_d["CategoricalNB"] = f1_score(test_y, pred_y,average='micro')


print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           1       0.59      0.73      0.65      2256
           2       0.52      0.34      0.41      2315
           3       0.43      0.49      0.46      2235
           4       0.43      0.42      0.42      2206

    accuracy                           0.49      9012
   macro avg       0.49      0.49      0.48      9012
weighted avg       0.49      0.49      0.48      9012



***7.4 Confusion Matrix***

In [ ]:
cf_matrix = confusion_matrix(y_true=test_y, y_pred=pred_y)
cf_matrix 

array([[1638,  118,  343,  157],
       [ 497,  781,  486,  551],
       [ 334,  271, 1086,  544],
       [ 312,  345,  616,  933]])

In [ ]:
y = ["Actual Severity 1", "Actual Severity 2", "Actual Severity 3", "Actual Severity 4"]
x =  ["Predicted Severity 1", "Predicted Severity 2", "Predicted Severity 3", "Predicted Severity 4"]

# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in cf_matrix]

# set up figure 
fig = ff.create_annotated_heatmap(cf_matrix, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.5,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.5,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        #textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(yaxis = dict(categoryorder = 'category descending'))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()




fig.write_html("Confusion_Matrix_Naive_Bayes.html")

***6.5 PR Curve***

In [ ]:
Y = label_binarize(valid_y, classes=[1, 2, 3, 4])

y_score = ranForest.predict_proba(valid_x)

precision_d["CategoricalNB"], recall_d["CategoricalNB"], _ = precision_recall_curve(Y.ravel(), y_score.ravel())
False_pos_Ratio["CategoricalNB"], True_pos_Ratio["CategoricalNB"], _ = roc_curve(Y.ravel(), y_score.ravel())

#7 Model Selection

is a method for selecting the most suitable or best model after the individual models are evaluated based on the required measures.

In [ ]:
accuracy_d

{'CategoricalNB': 0.4890556511124674,
 'Decision Tree': 0.6513621483659768,
 'Logistic Regression': 0.6289602174998613,
 'Random Forest': 0.7437163624257893,
 'neural Networks': 0.7111152410507202}

In [ ]:
f1_d

{'CategoricalNB': 0.4924545051043054,
 'Decision Tree': 0.6234500839403856,
 'Logistic Regression': 0.6271637816245007,
 'Random Forest': 0.7198180204172215,
 'neural Networks': 0.6943253874778748}

***7.1 Accuracy on Training set for each model***

In [ ]:
colors = ['#003f5c','#58508d','#bc5090','#ff6361','#ffa600'] 
fig = go.Figure()
fig.add_trace(go.Bar(x=list(accuracy_d.keys()), y=list(accuracy_d.values()),marker_color=colors))
fig.show()

fig.write_html("Accuracy_score_For_All_Models.html")



***7.2 F1 Score on Training set for each model***

In [ ]:
colors = ['#003f5c','#58508d','#bc5090','#ff6361','#ffa600'] 
fig = go.Figure()
fig.add_trace(go.Bar(x=list(f1_d.keys()), y=list(f1_d.values()),marker_color=colors))
fig.show()

fig.write_html("f1_score_For_All_Models.html")